# Loader for concept annotation task

The task is token classification into one or more of the following categories:

* Financial Flows
* Deforestation
* Vulnerable Groups
* Equity And Justice
* Challenges And Opportunities


In [2]:
import sys

!{sys.executable} -m pip install argilla

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 3.7 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached monotonic-1.6-py2.py3-none-any.whl (8.2 kB)
  Using cached Deprecated-1.2.13-py2.py3-none-any.whl (9.6 kB)
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Using cached wrapt-1.14.1-cp39-cp39-macosx_10_9_x86_64.whl (35 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 kB 2.5 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)

[notice] A new release of pip is available: 23.0 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import os
import random

from cpr_data_access.models import Dataset, BaseDocument
from dotenv import load_dotenv, find_dotenv
import argilla as rg
from tqdm.auto import tqdm
import spacy

load_dotenv(find_dotenv())
nlp = spacy.load("en_core_web_sm")

/Users/kalyan/Library/Caches/pypoetry/virtualenvs/experiment-template-Q6O6vM4r-py3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# User management is done at a workspace level

DATASET_NAME = "explorer-quality-testing"
TEXT_BLOCKS_PER_DOCUMENT = 2
settings = rg.TokenClassificationSettings(
    label_schema=[
        "Financial Flows",
        "Deforestation",
        "Vulnerable Groups",
        "Equity And Justice",
        "Challenges And Opportunities",
    ]
)

In [3]:
rg.init(workspace="gst", api_key=os.environ["ARGILLA_API_KEY"])

rg.configure_dataset(name=DATASET_NAME, settings=settings)

In [4]:
dataset = Dataset(document_model=BaseDocument).load_from_local(
    os.environ["DOCS_DIR_GST"], limit=None
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1142/1142 [01:08<00:00, 16.74it/s]


In [5]:
records = []

for document in tqdm(dataset.documents):
    if document.text_blocks is None:
        print(f"Skipping {document.document_id} as no text blocks")
        continue

    doc_metadata = document.dict(exclude={"text_blocks", "page_metadata"})

    # Randomly sample a fixed number of text blocks per document
    if len(document.text_blocks) <= TEXT_BLOCKS_PER_DOCUMENT:
        blocks = document.text_blocks
    else:
        blocks = random.sample(document.text_blocks, TEXT_BLOCKS_PER_DOCUMENT)

    for block in blocks:
        block_metadata = block.dict(exclude={"text"})
        block_text = block.to_string().replace("\n", " ").replace("  ", " ")

        records.append(
            rg.TokenClassificationRecord(
                text=block_text,
                tokens=[tok.text for tok in nlp(block_text)],
                metadata=doc_metadata | block_metadata,
                id=f"{block.text_block_id}_{document.document_id}",
            )
        )

  0%|▍                                                                                                                                                                                                                                            | 2/1142 [00:00<01:01, 18.40it/s]/Users/kalyan/Library/Caches/pypoetry/virtualenvs/experiment-template-Q6O6vM4r-py3.9/lib/python3.9/site-packages/argilla/client/models.py:76: UserWarning: Some metadata values could exceed the max length. For those cases, values will be truncated by keeping only the last 128 characters. You can configure this length in the server with the ARGILLA_METADATA_FIELD_LENGTH environment variable. Note that, setting this too high may lead to Elastic performance issues.
  warnings.warn(message, UserWarning)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [6]:
random.shuffle(records)

dataset_metadata = {
    "documents": [doc.document_id for doc in dataset.documents],
}

rg.log(
    records,
    name=DATASET_NAME,
    metadata=dataset_metadata,
)

/Users/kalyan/Library/Caches/pypoetry/virtualenvs/experiment-template-Q6O6vM4r-py3.9/lib/python3.9/site-packages/ri
ch/live.py:229: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

1916 records logged to 
]8;id=465155;http://ec2-3-9-155-244.eu-west-2.compute.amazonaws.com:6900/datasets/gst/explorer-quality-testing\http://ec2-3-9-155-244.eu-west-2.compute.amazonaws.com:6900/datasets/gst/explorer-quality-testing]8;;\

BulkResponse(dataset='explorer-quality-testing', processed=1916, failed=0)